In [1]:
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np

In [2]:
# Hyperparameters
learning_rate = 0.01
gamma = 0.5

In [3]:
class DQN(nn.Module):
    def __init__(self, cities, outputs):
        super(DQN, self).__init__()
        hidden = 128
        self.l1 = nn.Linear(cities, hidden)
        self.l2 = nn.Linear(hidden, outputs)

        # Overall reward and loss history
        self.reward_history = []
        self.loss_history = []
        self.reset()

    def reset(self):
        # Episode policy and reward history
        self.episode_actions = torch.Tensor([])
        self.episode_rewards = []

    def forward(self, x):
        model = torch.nn.Sequential(
            self.l1, nn.Dropout(p=0.5), nn.ReLU(), self.l2, nn.Softmax(dim=-1)
        )
        return model(x)

In [4]:
policy = DQN(12, 4)
optimizer = optim.Adam(policy.parameters(), lr=learning_rate)

In [5]:
def select_action(state):
    state = torch.from_numpy(state).type(torch.FloatTensor)
    print(state)
    action_probs = policy(state)
    numpy_probabilities = action_probs.detach().numpy()
    indices = numpy_probabilities.argsort()
    indices = np.flip(indices)
    avialible_actions = state[4:]
    for i in range(0, 4):
        number = avialible_actions[indices[i]*2]
        if number != -1.0:
            return indices[i]
    return

In [6]:
select_action(np.array([0.40999364, 0.10497416, 0.40999364, 0.10497416, 0.00869442, 0.81419548,
       0.44049597, 0.71548573, 0.24693963, 0.86145677, 0.31460646, 0.40467384]))

tensor([0.4100, 0.1050, 0.4100, 0.1050, 0.0087, 0.8142, 0.4405, 0.7155, 0.2469,
        0.8615, 0.3146, 0.4047])


0

In [7]:
def update_policy():
    R = 0
    rewards = []

    # Discount future rewards back to the present using gamma
    for r in policy.episode_rewards[::-1]:
        R = r + gamma * R
        rewards.insert(0, R)

    # Scale rewards
    rewards = torch.FloatTensor(rewards)
    rewards = (rewards - rewards.mean()) / \
        (rewards.std() + np.finfo(np.float32).eps)
    
    # Calculate loss
    loss = (torch.sum(torch.mul(policy.episode_actions, rewards).mul(-1), -1))

    # Update network weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Save and intialize episode history counters
    policy.loss_history.append(loss.item())
    policy.reward_history.append(np.sum(policy.episode_rewards))
    policy.reset()


In [8]:
points = np.array([[0.7008283 , 0.18339306],
       [0.40999364, 0.10497416],
       [0.23964594, 0.11362983],
       [0.00869442, 0.81419548],
       [0.44049597, 0.71548573]])

In [9]:
from math import sqrt

In [10]:
def calculate_distance(points):
    distance = 0.
    for i in range(1, len(points)):
        distance += sqrt((points[i-1][0] - points[i][0])**2) + (points[i-1][1] - points[i][1]**2)
        
    distance += sqrt((points[0][0] - points[len(points)-1][0])**2 + (points[0][1] - points[len(points)-1][1])**2)
    return distance

In [21]:
def main(episodes, points):
    transitions = {}
    for episode in range(episodes):
        cities =  np.reshape(points,(-1,2*len(points)))
        first_city =  points[0]
        first_c = np.reshape(first_city, (-1, 2))
        state = np.concatenate((first_c, cities), axis =1)[0]
        cost = 0
        done = False       
        transitions[episode] = [first_city]

        for step in range(0, len(points)):
            action = select_action(state)
            if action is None:
                break
            tmp_state = state
            tmp_state[2] = state[4 + action*2]
            tmp_state[3] = state[4 + action*2 + 1]
            tmp_state[4 + action*2] = -1.
            tmp_state[4 + action*2 + 1] = -1.
            print(tmp_state)
            next_state = tmp_state
            state = next_state
            transitions[episode].append(np.array(tmp_state[2], tmp_state[3]))

            
        return transitions[episode]  
        reward = R[state, action]
        transitions[episode].append(state)
        cost +=reward

             
        # Save reward
        policy.episode_rewards.append(reward)
        update_policy()

        if episode % 50 == 0:
            print('Episode {}\tCost: {:.2f}\tTransition:{}\tDistance:{}'.format(episode, cost,transitions[episode], calculate_distance(points,transitions[episode])))


In [22]:
def create_map(points):
    point_map = {}
    for i,p in enumerate(points):
        point_map[(p[0], p[1])] = i
    return point_map

In [23]:
main(10, points)

tensor([0.7008, 0.1834, 0.7008, 0.1834, 0.4100, 0.1050, 0.2396, 0.1136, 0.0087,
        0.8142, 0.4405, 0.7155])
[ 0.7008283   0.18339306  0.00869442  0.81419548  0.40999364  0.10497416
  0.23964594  0.11362983 -1.         -1.          0.44049597  0.71548573]
tensor([ 0.7008,  0.1834,  0.0087,  0.8142,  0.4100,  0.1050,  0.2396,  0.1136,
        -1.0000, -1.0000,  0.4405,  0.7155])
[ 0.7008283   0.18339306  0.44049597  0.71548573  0.40999364  0.10497416
  0.23964594  0.11362983 -1.         -1.         -1.         -1.        ]
tensor([ 0.7008,  0.1834,  0.4405,  0.7155,  0.4100,  0.1050,  0.2396,  0.1136,
        -1.0000, -1.0000, -1.0000, -1.0000])
[ 0.7008283   0.18339306  0.23964594  0.11362983  0.40999364  0.10497416
 -1.         -1.         -1.         -1.         -1.         -1.        ]
tensor([ 0.7008,  0.1834,  0.2396,  0.1136,  0.4100,  0.1050, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000])
[ 0.7008283   0.18339306  0.40999364  0.10497416 -1.         -1.
 -1.  

[array([0.7008283 , 0.18339306]),
 array(0.00869442),
 array(0.44049597),
 array(0.23964594),
 array(0.40999364)]